---
title: "Fuel Prices in Austria"
date: "2022-11-23"
categories: [python, altair, visualisation, economics]
description: An analysis of petrol prices in Austria across regions, fuel types and over time.
draft: true
---

In [1]:
import altair as alt
import pandas as pd
import requests 
from bs4 import BeautifulSoup


url = 'https://www.e-control.at'
r = requests.get(url + '/spritpreisrechner') 

soup = BeautifulSoup(r.content) 
download_url = url + soup.find_all("p", class_="ec-media-heading")[0].find_all(['a'])[0]['href']

data_raw = pd.read_excel(download_url, sheet_name='Median Diesel & Super')

diesel_index = data_raw[data_raw.iloc[:, 0] == 'DIESEL'].index[0]
super_index = data_raw[data_raw.iloc[:, 0] == 'SUPER'].index[0]

diesel_data = data_raw.iloc[(diesel_index + 1):(super_index - 3), :]
diesel_data.columns = data_raw.iloc[diesel_index].str.replace("-\n", "")
diesel_data = diesel_data.set_index('DIESEL')
diesel_data.columns.name = None
diesel_data.index.name = None

super_data = data_raw.iloc[(super_index + 1):, :]
super_data.columns = data_raw.iloc[super_index].str.replace("-\n", "")
super_data = super_data.set_index('SUPER')
super_data.columns.name = None
super_data.index.name = None

plot = pd.DataFrame()
plot['Diesel'] = diesel_data['Österreich']
plot['Super'] = super_data['Österreich']

plot = pd.melt(plot.reset_index(), id_vars=['index'], value_vars=plot.columns).dropna()
plot.columns = ['Date', 'Fuel Type', 'Price']

alt.Chart(plot).mark_line().encode(
    alt.X('Date', scale=alt.Scale(zero=False), title=None),
    alt.Y('Price', scale=alt.Scale(zero=False), title="Price (EUR/l)"),
    color=alt.Color(field='Fuel Type', scale=alt.Scale(scheme='category20'), title='Fuel Type'),
    tooltip=['Fuel Type', 'Date', 'Price']
).properties(
    width=550,
    height=300
)

alt.Chart(...)

In [2]:
plot = diesel_data['Österreich'].sub(super_data['Österreich']).to_frame('Premium')

plot = pd.melt(plot.reset_index(), id_vars=['index'], value_vars=plot.columns).dropna()
plot.columns = ['Date', 'Value', 'Premium']

line = alt.Chart(plot).mark_line().encode(
    alt.X('Date', scale=alt.Scale(zero=False), title=None),
    alt.Y('Premium', scale=alt.Scale(zero=False), title="Diesel premium (EUR/l)"),
    color=alt.Color(field='Value', scale=alt.Scale(scheme='category20'), legend=None),
    tooltip=['Date', 'Premium']
).properties(
    width=550,
    height=300
)

line

alt.Chart(...)

In [3]:
red = alt.Chart(plot).transform_filter(
    'datum.Premium>0'
).mark_area(
    color=alt.Gradient(
        gradient='linear',
        stops=[alt.GradientStop(color='white', offset=0),
               alt.GradientStop(color='darkred', offset=1)],
        x1=1, x2=1, y1=1, y2=-0.5
    )
).encode(
    alt.X('Date:T'),
    alt.Y('Premium:Q')
)

green = alt.Chart(plot).transform_filter(
    'datum.Premium<0'
).mark_area(
    color=alt.Gradient(
        gradient='linear',
        stops=[alt.GradientStop(color='darkgreen', offset=0),
               alt.GradientStop(color='white', offset=1)],
        x1=1, x2=1, y1=2, y2=0
    )
).encode(
    alt.X('Date:T'),
    alt.Y('Premium:Q')
)

line = alt.Chart(plot).mark_line().encode(
    alt.X('Date', scale=alt.Scale(zero=False), title=None),
    alt.Y('Premium', title="Diesel premium (EUR/l)"),
    color=alt.value('darkslategrey'),
    tooltip=['Date', 'Premium']
)

(green + red + line).properties(
    width=550,
    height=300
)

alt.LayerChart(...)

In [4]:
average = diesel_data.add(super_data).div(2)

plot = average.sub(average['Österreich'], axis=0).iloc[:, :-2]

plot = pd.melt(plot.reset_index(), id_vars=['index'], value_vars=plot.columns).dropna()
plot.columns = ['Date', 'Region', 'Premium']

alt.Chart(plot).mark_line().encode(
    alt.X('Date', scale=alt.Scale(zero=False), title=None),
    alt.Y('Premium', scale=alt.Scale(zero=False), title="Regional Premium (EUR/l)"),
    color=alt.Color(field='Region', scale=alt.Scale(scheme='category20')),
    tooltip=['Date', 'Premium']
).properties(
    width=550,
    height=300
)

alt.Chart(...)

In [5]:
plot = average.sub(average['Österreich'], axis=0).iloc[:, :-2].mean().reset_index()
plot.columns = ['Region', 'Premium']

alt.Chart(plot).mark_bar(size=30).encode(
    alt.X('Region', scale=alt.Scale(zero=False), title="Region", axis=alt.Axis(labelAngle=0), sort='-y'),
    alt.Y('Premium', title="Regional Premium (EUR/l)"),
    # color=alt.Color(field='Region', scale=alt.Scale(scheme='category20')),
    tooltip=['Region', alt.Tooltip('Premium', format=".2")]
).properties(
    width=550,
    height=300
)

alt.Chart(...)

In [6]:
days = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday', 'Sunday']
plot = average.groupby(average.index.weekday)["Österreich"].mean().reset_index()
plot.columns = ['Weekday', 'Premium']
plot.Premium -= plot.Premium.mean()
plot.Weekday = days

alt.Chart(plot).mark_bar(size=30).encode(
    alt.X('Weekday', scale=alt.Scale(zero=False), title="", axis=alt.Axis(labelAngle=0), sort=days),
    alt.Y('Premium', title="Weekday Premium (EUR/l)"),
    # color=alt.Color(field='Region', scale=alt.Scale(scheme='category20')),
    tooltip=['Weekday', alt.Tooltip('Premium', format=".2")]
).properties(
    width=550,
    height=300
)

alt.Chart(...)